In [1]:
# Parameters 
project = 'PowerBIDemo'
project_path = f'../src/{project}'

In [2]:
# Libraries
import pyfabricops as pf
pf.enable_notebook_logging()
pf.set_auth_provider('env') 

import json

In [3]:
# Relative paths
branch = pf.get_current_branch() 
branches_path = '../branches.json'
workspace_suffix = pf.get_workspace_suffix(path=branches_path)  
workspace_name = project + workspace_suffix
print(f'Branch: {branch}, Workspace: {workspace_name}') 

Branch: dev, Workspace: PowerBIDemo-DEV


In [4]:
# Retrienving the workspace_id from config.json in the project_path for better performance
config_path = f'{project_path}/config.json'
with open(config_path, 'r') as f:
  config_content = json.load(f)
  config = config_content[branch]  
  workspace_id = config[project]['workspace_config']['workspace_id'] 
print(f'Workspace ID: {workspace_id}')

Workspace ID: 1d2e9b3f-34d2-4a39-af39-6ebbc6b13d59


In [5]:
# Read the config.json retriving the parameters for the semantic model
with open(config_path, 'r') as f:
	config_content = json.load(f)
config = config_content[branch]
semantic_models_config = config[project]['semantic_models']

In [6]:
# Replace the placeholders with the actual values
import glob 
import re

for semantic_model_path in glob.glob(f'{project_path}/**/*.SemanticModel', recursive=True):
	print(f'Processing semantic model: {semantic_model_path}')
	semantic_model_name = semantic_model_path.replace('\\', '/').split('/')[-1].split('.SemanticModel')[0] 
	print(f'Deploying semantic model name: {semantic_model_name}')  

	# Retrieving the semantic model config
	semantic_model_config = semantic_models_config[semantic_model_name]
	semantic_model_parameters = semantic_model_config['parameters']
	# Define your new values here
	new_server = semantic_model_parameters['Server']['Value']
	new_database = semantic_model_parameters['Database']['Value']
	new_calendar_workspace_id = semantic_model_parameters['CalendarWorkspaceId']['Value']
	new_calendar_dataflow_id = semantic_model_parameters['CalendarDataflowId']['Value']

	# Read the expressions file
	expressions_path = f'{semantic_model_path}//definition/expressions.tmdl'
	with open(expressions_path, 'r') as f:
		expressions = f.read()

	# Dictionary with variable mappings
	variables_to_replace = {
		'Server': new_server,
		'Database': new_database,
		'CalendarWorkspaceId': new_calendar_workspace_id,
		'CalendarDataflowId': new_calendar_dataflow_id
	}
	
	# Start with original expressions
	expressions_updated = expressions

	# Replace each variable using regex
	for variable_name, new_value in variables_to_replace.items():
		# Pattern to match: VariableName = "any_value"
		pattern = rf'{variable_name}\s*=\s*"[^"]*"'
		replacement = f'{variable_name} = "{new_value}"'
		expressions_updated = re.sub(pattern, replacement, expressions_updated)
		print(f"Replaced {variable_name} with: {new_value}")

	# Write the updated expressions back to the file
	with open(expressions_path, 'w', encoding='utf-8') as f:
		f.write(expressions_updated)


Processing semantic model: ../src/PowerBIDemo\workspace\Main\Sales.SemanticModel
Deploying semantic model name: Sales
Replaced Server with: PEZZOTT
Replaced Database with: AdventureWorksDW2022-DEV
Replaced CalendarWorkspaceId with: 1d2e9b3f-34d2-4a39-af39-6ebbc6b13d59
Replaced CalendarDataflowId with: d7821848-dc82-4558-9507-64636f4c296c
